In [1]:
# Import required libraries
import torch
import torchvision.transforms as transforms
import requests
from PIL import Image
from transformers import CLIPProcessor, CLIPModel


# Load CLIP processor
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
# Load pre-trained model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

print("Loaded model and processor")

Loaded model and processor


In [2]:
# Set the model to evaluation mode
model.eval()

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [3]:
# Define preprocessing transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize input image
    transforms.ToTensor(),  # Convert PIL image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize image
])


In [5]:
# Load and preprocess input data
#input_image = Image.open('path_to_input_image.jpg')

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
input_image = Image.open(requests.get(url, stream=True).raw)
input_tensor = transform(input_image).unsqueeze(0)  # Add batch dimension

In [12]:
# Perform inference
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

# Interpret output (depends on the task and model)
# Example for classification:
probabilities = torch.softmax(outputs, dim=1)
predicted_class = torch.argmax(probs, dim=1).item()

# Example for object detection:
# Post-process bounding box predictions, draw boxes on image, etc.
print('Predicted class:', predicted_class)


TypeError: softmax() received an invalid combination of arguments - got (CLIPOutput, dim=int), but expected one of:
 * (Tensor input, int dim, torch.dtype dtype, *, Tensor out)
 * (Tensor input, name dim, *, torch.dtype dtype)
